In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

import transitions
from collections import deque
import plotly.express as px
from IPython.display import display, Markdown
# display(Markdown(trial_results.to_markdown()))

In [2]:
laser_iv = np.array([3.339754422650176, 1.003425703613611])
laser_li = np.array([0.3061936769909521, -0.0056601769911848775])

In [3]:
class Laser(object):
   """
   A class to represent a Laser device.

   Attributes
   ----------
   i_drive : float
      The drive current of the laser.
   tbk : float
      The backside temperature of the laser.
   rjc : float
      The junction-case thermal resistance.
   tbk0 : float
      The initial backside temperature.
   i_drive0 : float
      The initial drive current.
   laser_wavelength : float
      The wavelength of the laser.
   laser_iv : list
      The current-voltage characteristics of the laser.
   laser_li : list
      The light-current characteristics of the laser.
   nm_degC : float
      The wavelength shift per degree Celsius.

   Methods
   -------
   _device_characteristics(laser_iv, laser_li):
      Initializes the device characteristics.
   device_state(i_drive, tbk):
      Returns the state of the device given the drive current and backside temperature.
   """
   def __init__(
      self,
      i_drive,
      tbk,
      rjc=45,
      tbk0 = 35,
      i_drive0 = 0.14) -> None:
      
      self.i_drive = i_drive
      self.tbk = tbk
      self.rjc = rjc
      self.tbk0 = tbk0
      self.i_drive0 = i_drive0
      self.laser_wavelength = 1310e-9
      self._device_characteristics()
      self._init_device_state = self.device_state(self.i_drive0, self.tbk0)
      self._operating_device_state = self.device_state(self.i_drive, self.tbk)
      self._operating_device_state["lambda"] = self._init_device_state["lambda"] + (self._operating_device_state["Tj"] - self._init_device_state["Tj"])*self.nm_degC * 0.1e-9
   
   def _device_characteristics(self, laser_iv=laser_iv, laser_li=laser_li):
      self.laser_iv = laser_iv
      self.laser_li = laser_li
      self.nm_degC = 0.1
      
   def __repr__(self) -> str:
      return f"Laser(i_drive={self.i_drive}, tbk={self.tbk})"
   
   def __str__(self) -> str:
      return self.__repr__()
   
   def __hash__(self) -> int:
      return hash((self.i_drive, self.tbk))
   
   def device_state(self, i_drive, tbk):
      v_drive = self.laser_iv[0]*i_drive + self.laser_iv[1]
      ele_power = v_drive*i_drive
      opt_power = i_drive * self.laser_li[0] + self.laser_li[1]
      wpe = opt_power/ele_power
      heat_diss = ele_power - opt_power
      Tj = tbk + self.heat_diss*self.rjc
   
      device_state = {
         "v_drive": v_drive,
         "i_drive": i_drive,
         "ele_power": ele_power,
         "opt_power": opt_power,
         "heat_diss": heat_diss,
         "wpe": wpe,
         "tbk": tbk,
         "Tj": Tj,
         "lambda": self.laser_wavelength
      }
      
      return device_state
      
   @property
   def init_device_state(self):
      return self._init_device_state
   
   @property
   def operating_device_state(self):
      return self._operating_device_state
   
   @property
   def drive_current(self):
      return self.i_drive
   
   @drive_current.setter
   def drive_current(self, i_drive):
      self.i_drive = i_drive
      self._operating_device_state = self.device_state(i_drive, self.tbk)
      
   @property
   def backside_temp(self):
      return self.tbk
   
   @backside_temp.setter
   def backside_temp(self, tbk):
      self.tbk = tbk
      self._operating_device_state = self.device_state(self.i_drive, tbk)
      self._operating_device_state["lambda"] = self._init_device_state["lambda"] + (self._operating_device_state["Tj"] - self._init_device_state["Tj"])*self.nm_degC * 0.1e-9

In [4]:
class MUX(object):
   """
   A class to represent a Multiplexer (MUX) device.

   Attributes
   ----------
   center_wavelength : float
      The center wavelength of the MUX.
   laser_wavelength : float
      The wavelength of the laser.
   tbk : float
      The backside temperature of the MUX.
   tbk0 : float
      The initial backside temperature.
   nm_degC : float
      The wavelength shift per degree Celsius.
   ave_noise_floor : float
      The average noise floor.
   il_mux_dB : float
      The insertion loss of the MUX in dB.
   il_demux_dB : float
      The insertion loss of the DEMUX in dB.
   mux_bw_nm : float
      The bandwidth of the MUX in nm.
   demux_bw_nm : float
      The bandwidth of the DEMUX in nm.

   Methods
   -------
   _device_characteristics():
      Initializes the device characteristics.
   device_state(tbk):
      Returns the state of the device given the backside temperature.
   """
   
   def __init__(
      self,
      center_wavelength,
      laser_wavelength,
      tbk,
      tbk0 = 45
   ):
      self.center_wavelength = center_wavelength
      self.laser_wavelength = laser_wavelength
      self.tbk = tbk
      self.tbk0 = tbk0
      self._device_characteristics()
      self._init_device_state = self.device_state(tbk=self.tbk0)
      self._operating_device_state = self.device_state(tbk=self.tbk)
      
   def __repr__(self) -> str:
      return f"MUX(center_wavelength={self.center_wavelength}, laser_wavekngth={self.laser_wavelength}, tbk={self.tbk})"
   
   def __str__(self) -> str:
      return self.__repr__()
   
   def _device_characteristics(self):
      self.nm_degC = 0.01
      self.ave_noise_floor = -40
      self.il_mux_dB = -4.45
      self.il_demux_dB = -20.45
      self.mux_bw_nm = 0.63
      self.demux_bw_nm = 0.27
   
      
   def device_state(self, tbk):
      noise_floor = 10 ** (np.random.normal(self.ave_noise_floor, 2))
      
      self.center_wavelength = self.center_wavelength + (tbk - self.tbk0)*self.nm_degC * 1e-9
      
      il_mux_lin = 10**(self.il_mux_dB/10)
      il_demux_lin = 10**(self.il_demux_dB/10)
      
      il_mux_output = il_mux_lin * np.exp(-((self.center_wavelength - self.laser_wavelength)/self.mux_bw_nm/1e-9)**2/(4.343))
      il_demux_output = il_demux_lin * np.exp(-((self.center_wavelength - self.laser_wavelength)/self.demux_bw_nm/1e-9)**2/(4.343))
      
      if il_mux_output < 10**(-self.ave_noise_floor/10):
         il_mux_output = noise_floor
      if il_demux_output < 10**(-self.ave_noise_floor/10):
         il_demux_output = noise_floor
         
      device_state = {
         "center_wavelength": self.center_wavelength,
         "laser_wavelength": self.laser_wavelength,
         "tbk": tbk,
         "il_mux_output": il_mux_output,
         "il_demux_output": il_demux_output
      }
   
      return device_state
   
   @property
   def init_device_state(self):
      return self._init_device_state
   
   @property
   def operating_device_state(self):
      return self._operating_device_state
   
   @property
   def center_wavelength(self):
      return self._center_wavelength
   
   @property
   def laser_wavelength(self):
      return self._laser_wavelength
   
   @laser_wavelength.setter
   def laser_wavelength(self, laser_wavelength):
      self._laser_wavelength = laser_wavelength
      self._operating_device_state = self.device_state(self.tbk)
      
   @property
   def backside_temp(self):
      return self.tbk
   
   @backside_temp.setter
   def backside_temp(self, tbk):
      self.tbk = tbk
      self._operating_device_state = self.device_state(tbk)

In [ ]:
class LasaerModule(object):
   def __init__(self, laser, mux):
      self.laser = laser
      self.mux = mux
      self._init_device_state = self.device_state()
      self._operating_device_state = self.device_state()